## Install

### Install yolov5

In [2]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
# !git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

# !pip install -qr requirements.txt
# !pip install -q roboflow

c:\Users\Ewen\Documents\Ewen\OsloMet\AiResearch\MultipleClassMultipleObjectDetection\yolov5


fatal: destination path 'yolov5' already exists and is not an empty directory.


### Verify configuration

In [1]:
import torch
if not torch.cuda.is_available():
  print("WARNING: GPU not available... Running on CPU")
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 1.12.1+cu116 _CudaDeviceProperties(name='NVIDIA GeForce RTX 3060 Laptop GPU', major=8, minor=6, total_memory=6143MB, multi_processor_count=30)


### Install datasets

In [3]:
from roboflow import Roboflow
rf = Roboflow(api_key="API_KEY")
project = rf.workspace("WORKSPACE").project("PROJECT")
dataset = project.version(1).download("DOWNLOAD")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to bcdd-1 in yolov5pytorch:: 100%|██████████| 1536/1536 [00:00<00:00, 1857.58it/s]


## Config

In [5]:
DATASET_CONFIG_FILE = f"{dataset.location}/data.yaml"
MODEL_CONFIG_FILE = f"models/yolov5s.yaml"
TRAINING_RESULTS_FOLDER_NAME = "training_results"
TRAINING_RESULTS_PATH = f'runs/train/{TRAINING_RESULTS_FOLDER_NAME}'
TEST_RESULTS_FOLDER_NAME = "test_results"
TEST_RESULTS_PATH = f'runs/detect/{TEST_RESULTS_FOLDER_NAME}'

### Display dataset config

In [6]:
with open(DATASET_CONFIG_FILE, "r") as f:
    print(f.read())

names:
- Platelets
- RBC
- WBC
nc: 3
train: bcdd-1/train/images
val: bcdd-1/valid/images



In [7]:
import yaml
num_classes = 1
with open(DATASET_CONFIG_FILE, 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])
print("Classes:", num_classes)

Classes: 3


### Personnalize model config

In [8]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [9]:
%%writetemplate {MODEL_CONFIG_FILE}

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [10]:
with open(MODEL_CONFIG_FILE, "r") as f:
    print(f.read())


# parameters
nc: 3  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-

## Train

In [11]:
# print(f"!python train.py --img 416 --batch 4 --epochs 100 --patience 5 --data {DATASET_CONFIG_FILE} --cfg {MODEL_CONFIG_FILE} --name {TRAINING_RESULTS_FOLDER_NAME}  --cache")
!python train.py --img 416 --batch 16 --epochs 100 --patience 5 --data {DATASET_CONFIG_FILE} --cfg {MODEL_CONFIG_FILE} --name {TRAINING_RESULTS_FOLDER_NAME}  --cache

!python train.py --img 416 --batch 4 --epochs 100 --patience 5 --data c:\Users\Ewen\Documents\Ewen\OsloMet\AiResearch\MultipleClassMultipleObjectDetection\yolov5\bcdd-1/data.yaml --cfg models/yolov5s.yaml --name training_results  --cache


## Evaluate

In [ ]:
from IPython.display import Image
Image(filename=f'{TRAINING_RESULTS_PATH}/results.png', width=1000)  # view results.png

In [ ]:
print("GROUND TRUTH TRAINING DATA:")
Image(filename=f'{TRAINING_RESULTS_PATH}/val_batch0_labels.jpg', width=900)

In [ ]:
print("PREDICTED DATA:")
Image(filename=f'{TRAINING_RESULTS_PATH}/val_batch0_pred.jpg', width=900)

## Test

In [ ]:
!python detect.py --weights '{TRAINING_RESULTS_PATH}/weights/best.pt' --img 416 --conf 0.4 --source {dataset.location}/test/images --name {TEST_RESULTS_FOLDER_NAME}

In [ ]:
import glob
from IPython.display import Image, display

for imageName in glob.glob(f'{TEST_RESULTS_PATH}/*.jpg'): #assuming JPG
    display(Image(filename=imageName))
    print("\n")